In [30]:
import pandas as pd
import pixiedust

import requests

In [31]:
df = pd.read_csv('DB_ready.csv')

In [36]:
config = dict(
    username="2660e8fa-aac2-45ef-81ba-217967728ad2-bluemix", 
    password="9cd1b00e6b6927e663bfa0e2e9d677e163378a7c98e09bfb598dd74a4439fc7a",
    database="cfc_patient_db")

In [37]:
url = "https://{username}:{password}@{username}.cloudant.com/{database}/_all_docs".format(**config)

In [42]:
auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
r = requests.get(url= url )

In [47]:
print(r.text)

{"total_rows":1112,"offset":0,"rows":[
{"id":"5354af5c52e35b07bd695d11ec72903b","key":"5354af5c52e35b07bd695d11ec72903b","value":{"rev":"1-d2d566152570b7ac8e667c42b274f9c6"}},
{"id":"5354af5c52e35b07bd695d11ec729299","key":"5354af5c52e35b07bd695d11ec729299","value":{"rev":"1-a831257d1ce43066708073378f35e3e9"}},
{"id":"5354af5c52e35b07bd695d11ec7297f7","key":"5354af5c52e35b07bd695d11ec7297f7","value":{"rev":"1-3680ce6a67107b741c251270102fe9a4"}},
{"id":"5354af5c52e35b07bd695d11ec7298be","key":"5354af5c52e35b07bd695d11ec7298be","value":{"rev":"1-6dbfd65e42b3bcdb34db7d0e8e8d6702"}},
{"id":"5354af5c52e35b07bd695d11ec72a0e8","key":"5354af5c52e35b07bd695d11ec72a0e8","value":{"rev":"1-57c50bb5e0787a0c9d5c714f19cd3999"}},
{"id":"5354af5c52e35b07bd695d11ec72af3f","key":"5354af5c52e35b07bd695d11ec72af3f","value":{"rev":"1-833bbecc4711e1f336974db36e51079d"}},
{"id":"5354af5c52e35b07bd695d11ec72af4f","key":"5354af5c52e35b07bd695d11ec72af4f","value":{"rev":"1-e1a8ffd00797310677ba82de94a9dd21"}},
{"